## Theoretical Concept

Naive Bayes classifier is a very efficient classifier which is widely used when there are multiple classes. It is based on the <a href="https://www.wikiwand.com/en/Bayes%27_theorem"> Bayes Theorem </a>

<center>$ P(Class | Data) = \frac{P(Data | Class) * P(Class)}{P(Data)}$</center>

where:
 - P(Class | Data) is the probability of a class given the data.(<a href="https://www.wikiwand.com/en/Posterior_probability"> Posterior Probability </a>)
 - P(Data | Class) is the probability of the data given a particular class.

## Importing modules


In [320]:
from random import seed
from random import randrange
from math import sqrt
from math import exp
from math import pi
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings( "ignore" )
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score


## Load the data into a dataframe

In [321]:
df = pd.read_csv('Data/iris.csv', names = ['sepal_length','sepal_width','petal_length','petal_width','class'])

In [322]:
display(df)
print(df.info())
print(df.describe())
print(set(df['class']))

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB
None
       sepal_length  sepal_width  petal_length  petal_width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.054000      3.758667     1.198667
std        0.828066     0.433594      1.764420     0.763161
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.50000

We need to enumerate the categories of the class to an integer value for our model.

In [323]:
map_dict = {'Iris-setosa':0, 'Iris-virginica':1, 'Iris-versicolor':2}
df['class'] = df['class'].map(map_dict)
display(df)

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,1
148,6.2,3.4,5.4,2.3,1


In [324]:
X = [list(df.iloc[i,:-1]) for i in range(len(df))]
Y = list((df.iloc[:,-1]))

X_train,  X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
print(X_train[0])
print(X_test[0])

[5.5, 2.4, 3.7, 1.0]
[6.5, 2.8, 4.6, 1.5]


In [325]:
def separate_by_class(X):
    seperated = {}
    for i in range(len(X)):
        if Y[i] in seperated:
            #seperated[Y[i]] = seperated[Y[i]] + X[i]
            seperated[Y[i]].append(X[i])
        else:
            seperated[Y[i]] = [X[i]]
    return seperated


In [326]:
def summarize_dataset(dataset):
    summaries = [(np.mean(column), np.std(column), len(column)) for column in zip(*dataset)]
    del(summaries[-1])
    return summaries
summaries = summarize_dataset(X)

print(summaries)

[(5.843333333333334, 0.8253012917851409, 150), (3.0540000000000003, 0.4321465800705435, 150), (3.758666666666666, 1.7585291834055212, 150)]


In [327]:
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = dict()
    for class_value, rows in separated.items():
        summaries[class_value] = summarize_dataset(rows)
    return summaries


In [328]:
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    probabilities = dict()
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities

In [329]:
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

In [330]:
summaries = summarize_by_class(X_train)
probabilities = calculate_class_probabilities(summaries, X_test[0])
print(probabilities)

{0: 0.03146056931668375, 2: 0.02027871279053155, 1: 0.004503113297867268}
